In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from urllib.parse import urlparse, parse_qsl
import matplotlib
matplotlib.use('pgf')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import django
import pickle
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize

pgf_with_latex = {
    "text.usetex": True,            # use LaTeX to write all text
    "pgf.rcfonts": False,           # Ignore Matplotlibrc
    "text.latex.unicode": True,
    "pgf.preamble": [
        #r"\usepackage[utf8x]{inputenc}",
        r"\usepackage{xcolor}"
    ],
    "pgf.texsystem" : "xelatex",
    "figure.figsize": [3.5,5]
}
matplotlib.rcParams.update(pgf_with_latex)

In [23]:
ars = AR.objects.filter(ar__gt=0).order_by('ar').values('name','start','end')

for ar in ars:
    ar["years"] = f'{ar["start"]}-{ar["end"]}'
    if ar["name"] == "AR6":
        ar["years"] = f'{ar["start"]}-'

ardf = pd.DataFrame.from_dict(list(ars))[['name','years']]

ardf.to_latex("../tables/ar_years.tex",index=False)
#ardf['years'] = f"-{ardf['start']}"
ardf.head(6)


,name,years
0,AR1,1988-1989
1,AR2,1990-1994
2,AR3,1996-2000
3,AR4,2001-2006
4,AR5,2007-2013
5,AR6,2014-


In [37]:
with open("../tables/sizes_X.pickle", "rb") as f:
    X = pickle.load(f)
    
with open("../tables/vecs.pickle", "rb") as f:
    vecs = pickle.load(f)

[<625x1380 sparse matrix of type '<class 'numpy.int64'>'
 	with 9191 stored elements in Compressed Sparse Row format>,
 <7623x12409 sparse matrix of type '<class 'numpy.int64'>'
 	with 478533 stored elements in Compressed Sparse Row format>,
 <16395x20453 sparse matrix of type '<class 'numpy.int64'>'
 	with 1270681 stored elements in Compressed Sparse Row format>,
 <34510x32644 sparse matrix of type '<class 'numpy.int64'>'
 	with 2818379 stored elements in Compressed Sparse Row format>,
 <117758x67064 sparse matrix of type '<class 'numpy.int64'>'
 	with 10077910 stored elements in Compressed Sparse Row format>,
 <128266x74196 sparse matrix of type '<class 'numpy.int64'>'
 	with 11572877 stored elements in Compressed Sparse Row format>]

In [ ]:
with open("../tables/sizes_f_X.pickle", "rb") as f:
    f_X = pickle.load(f)
    
with open("../tables/f_vecs.pickle", "rb") as f:
    f_vecs = pickle.load(f)

In [93]:
all_vocab = set()
ignore_vocab = set([
    'tildeo','2degreesc','5degreesc','1degreesc','0degreesc','5degrees',
    'to10','3degreesc','4degreesc','5parts','8degreesc'
])
new_vocabs = []
for i, vec in enumerate(vecs):
    x = X[i]
    vocab = vec.vocabulary_
    sum_words = x.sum(axis=0).tolist()[0]
    new_vocab = [(x[0],sum_words[x[1]]) for x in vocab.items() if x[0] not in all_vocab and x[0] not in ignore_vocab]
    new_vocab =sorted(new_vocab, key = lambda x: x[1], reverse=True)
    all_vocab = all_vocab | set(vec.get_feature_names()) - ignore_vocab
    new_vocabs.append(new_vocab[:10])
    print(new_vocab[:20])
    print('\n')

tfmt = " ".join(["p{2cm}" for x in range(7)])
tfmt = " ".join(["l" for x in range(7)])
tfmt = "l " + " ".join(["p{1.8cm}" for x in range(6)])
hrow = "&" + " & ".join([f"\\textbf{{{x.name}}}" for x in AR.objects.filter(ar__gt=0).order_by('ar')]) + "\\\\ \hline"
drow = "\\textbf{Documents} &" +" & ".join([str(x.shape[0]) for x in X]) + "\\\\ \n"
wrow = "\\textbf{Words} &" +" & ".join([str(x.shape[1]) for x in X]) + "\\\\ \n"
#nwrow = "\\textbf{Words} &" + " & ".join([str(x) for x in new_vocabs]) 
nwrows = "\\textbf{New words}" + "\\\\".join([" & " + " & ".join([f"{x[i][0]} ({x[i][1]})" for x in new_vocabs]) for i in range(8)])
print(hrow) 

with open("../tables/growth_table.tex","w") as f:
    f.write("\\begin{tabular}{" + tfmt + "} \n")
    f.write(hrow)
    f.write(drow)
    f.write(wrow)
    f.write(nwrows)
    f.write("\n\\end{tabular}")
## MMMs = Mixed matrix membrane
nwrow

[('change', 296), ('climate', 262), ('model', 168), ('effect', 160), ('co2', 156), ('atmospheric', 152), ('climatic', 133), ('global', 131), ('greenhouse', 109), ('ocean', 106), ('temperature', 104), ('carbon', 95), ('ice', 84), ('year', 83), ('surface', 82), ('use', 81), ('sea', 80), ('water', 77), ('increase', 74), ('time', 71)]


[('loss', 552), ('efficiency', 515), ('mol', 439), ('ambient', 417), ('coal', 404), ('photosynthetic', 393), ('concern', 381), ('chamber', 353), ('demonstrate', 351), ('consumption', 305), ('product', 299), ('throughout', 297), ('way', 297), ('combustion', 293), ('unit', 287), ('positive', 285), ('oil', 285), ('deltac', 283), ('east', 265), ('demand', 258)]


[('downscaling', 197), ('degreesc', 145), ('ncep', 130), ('otcs', 87), ('nee', 87), ('fco', 80), ('hadcm2', 78), ('dtr', 75), ('annex', 71), ('earthworm', 60), ('boreas', 56), ('oeschger', 54), ('cdm', 51), ('opt', 50), ('reanalyses', 50), ('amip', 49), ('bracken', 49), ('msw', 48), ('bayesian', 48), (

"\\textbf{Words} &[('change', 296), ('climate', 262), ('model', 168), ('effect', 160), ('co2', 156), ('atmospheric', 152), ('climatic', 133), ('global', 131), ('greenhouse', 109), ('ocean', 106)] & [('loss', 552), ('efficiency', 515), ('mol', 439), ('ambient', 417), ('coal', 404), ('photosynthetic', 393), ('concern', 381), ('chamber', 353), ('demonstrate', 351), ('consumption', 305)] & [('downscaling', 197), ('degreesc', 145), ('ncep', 130), ('otcs', 87), ('nee', 87), ('fco', 80), ('hadcm2', 78), ('dtr', 75), ('annex', 71), ('earthworm', 60)] & [('sres', 217), ('5degreesc', 200), ('2degreesc', 125), ('5degrees', 111), ('1degreesc', 109), ('0degreesc', 103), ('to10', 98), ('petm', 95), ('3degreesc', 88), ('4degreesc', 87)] & [('biochar', 1752), ('redd', 1058), ('cmip5', 656), ('cmip3', 569), ('wrf', 334), ('mofs', 288), ('sdm', 283), ('gosat', 281), ('mof', 271), ('biochars', 252)] & [('mmms', 192), ('c3n4', 132), ('cop21', 107), ('cmip6', 104), ('zika', 75), ('brgdgts', 71), ('twitter'

In [77]:
x = new_vocabs[0]
x[0][0]

'change'

In [5]:
## parse the bible, turn it into same format
import codecs
import re
bible = []
i=0
clines=""
corpus = []
lchapter="none"
with codecs.open('/home/galm/projects/big_literature/py/Martin_Luther_Uebersetzung_1912.txt',encoding = "ISO-8859-1") as ml:
    for line in ml:
        chapter = line.split(':')[0]
        try:
            verse = re.search(".*([0-9]+\:[0-9]*)(.*)",line).group(2).strip()
        except:
            corpus.append(str(clines))
            break
        if lchapter!=chapter and i > 0:
            corpus.append(str(clines))
            clines=verse            
        else:
            clines+=" "+verse
            #print('\n')
        lchapter = chapter
        i+=1
        
X_y = c_vectorizer.fit_transform(corpus)
X.append(X_y)

In [20]:
ind = np.arange(1)

def vvrect(X,col_x,m):
    return plt.bar(ind+X.shape[1]/2,X.shape[0],width=X.shape[1],facecolor=col_x,edgecolor='black',alpha=m)#,alpha=m)

means = [x.getnnz(0).mean()/x.shape[0]*100 for x in X]
nmeans = normalize([means])[0]

crange = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc']

crange = ["#D53E4F", "#FC8D59", "#FEE08B", "#E6F598", "#99D594", "#3288BD"]

crange = ["#8dd3c7","#ffffb3","#bebada","#fb8072","#80b1d3","#fdb462","grey"]

years = [x.name for x in ars] + ["Bible"]

fig = plt.figure()

plots = []
lvalues = []
for x in [5,4,3,2,1,0,6]:
    a = (nmeans[x]+0.3)/1.3
    #a = nmeans[x]
    vvrect(X[x], "None",1)
    vvrect(X[x], "white",1)
    p = vvrect(X[x], crange[x] ,a)
    plots.append(p[0])
    lvalues.append(years[x] + ": " + str(round(means[x],2)))
    print(crange[5-x])
    print(a)
    
plt.legend(plots,lvalues,title='% of documents mean\nterm appears in')
    
plt.axis('equal')
plt.ylabel('Documents')
plt.xlabel('Words')
fig.patch.set_facecolor('#f0f0f0')    
#plt.tight_layout()

plt.savefig(
    '../plots/literature_size/volume_variety.pdf',
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
)

plt.show()


#8dd3c7
0.26106256071014744
#ffffb3
0.26469409500636126
#bebada
0.29105686163912314
#fb8072
0.33736081753188196
#80b1d3
0.41035679812605236
#fdb462
0.43180147447054457
grey
0.9392784910599727


In [7]:
tab = pd.DataFrame([(x.shape[0],x.shape[1]) for x in X]).rename({0:'Documents',1:'Terms'},axis="columns")
tab.index = years
tab['Average Document score per Term'] = means
tab

,Documents,Terms,Average Document score per Term
AR1,1848,2834,0.580116
AR2,6941,13279,0.518233
AR3,18728,26657,0.307590
AR4,44000,49955,0.173971
AR5,108277,92369,0.097896
AR6,128357,108102,0.087417
Bible,1189,10662,2.044534


In [8]:

shape = [(x+6000)/10000 for x in list(X[5].shape)[::1]]

#plt.rcParams["figure.figsize"] = tuple([round(x) for x in shape[::1]])

#plt.rcParams["figure.figsize"] = (9.5,13)

xs = [5,4,3,2,1,0,6]

for ar in AR.objects.all():
    
    iprs = IPCCRef.objects.filter(ar=ar).count()
    
    plots = []
    lvalues = []

    ar_xs = [x for x in xs if ar.ar > x ]
    print(ar_xs)
    #continue
    for x in ar_xs:
        a = (nmeans[x]+0.1)/1.1
        #a = nmeans[x]
        vvrect(X[x], "None",1)
        vvrect(X[x], "white",1)
        p = vvrect(X[x], crange[x] ,a)
        plots.append(p[0])
        lvalues.append(years[x] + ": " + str(round(means[x],2)))
        ar = AR.objects.get(ar=x+1)
        iprs = IPCCRef.objects.filter(ar=ar).count()
        xtent = shape[1]*10000+5000+1000
        #plt.axhline(iprs,xmin=1/xtent*1000,xmax=1/xtent*(X[x].shape[1]+1000))
    
    plt.legend(plots,lvalues,title='% of documents mean term appears in')
    
    #plt.axis('equal')

    #plt.ylim((-1000,130000))
    #plt.ylim(ymin=-1000)
    #plt.xlim((-1000,100000))
    plt.xlim((-1000,shape[1]*10000+5000))
    plt.ylim((-1000,shape[0]*10000+5000))
    #plt.ylim(ymax=shape[0]*10000)
    #plt.xlim(xmax=shape[1]*10000)
    plt.ylabel('Documents')
    plt.xlabel('Words')
    plt.savefig('../plots/literature_size/volume_variety_{}.pdf'.format(ar.name),bbox_inches='tight')
    
    print(plt.xlim())

    plt.show()


[]
(-1000.0, 119102.0)
[0]
(-1000.0, 119102.0)
[1, 0]
(-1000.0, 119102.0)
[2, 1, 0]
(-1000.0, 119102.0)
[3, 2, 1, 0]
(-1000.0, 119102.0)
[4, 3, 2, 1, 0]
(-1000.0, 119102.0)
[5, 4, 3, 2, 1, 0]
(-1000.0, 119102.0)


In [9]:
for ar in ars:
    plots = []
    lvalues = []

    ar_xs = [x for x in xs if ar.ar > x or x ==6]
    print(ar_xs)
    #continue
    for x in ar_xs:
        a = (nmeans[x]+0.1)/1.1
        #a = nmeans[x]
        vvrect(X[x], "None",1)
        vvrect(X[x], "white",1)
        p = vvrect(X[x], crange[x] ,a)
        plots.append(p[0])
        #lvalues.append(years[x] + ": " + str(round(X[x].mean()*1000000/X[x].sum(),5)))
        lvalues.append(years[x] + ": " + str(round(means[x],2)))
        print(crange[5-x])
        print(a)
    
    plt.legend(plots,lvalues,title='% of documents mean term appears in')
    
    #plt.axis('equal')

    #plt.ylim((-1000,130000))
    #plt.ylim(ymin=-1000)
    #plt.xlim((-1000,100000))
    plt.xlim((-1000,shape[1]*10000+5000))
    plt.ylim((-1000,shape[0]*10000+5000))
    #plt.ylim(ymax=shape[0]*10000)
    #plt.xlim(xmax=shape[1]*10000)
    plt.ylabel('Documents')
    plt.xlabel('Words')
    plt.savefig('../plots/literatures_size/volume_variety_bible_{}.pdf'.format(ar.name),bbox_inches='tight')
    
    print(plt.xlim())

    plt.show()



[0, 6]
#fdb462
0.32849265164700725
grey
0.9282382167072406
(-1000.0, 119102.0)
[1, 0, 6]
#80b1d3
0.30314894323988
#fdb462
0.32849265164700725
grey
0.9282382167072406
(-1000.0, 119102.0)
[2, 1, 0, 6]
#fb8072
0.21688096617404234
#80b1d3
0.30314894323988
#fdb462
0.32849265164700725
grey
0.9282382167072406
(-1000.0, 119102.0)
[3, 2, 1, 0, 6]
#bebada
0.1621581092098728
#fb8072
0.21688096617404234
#80b1d3
0.30314894323988
#fdb462
0.32849265164700725
grey
0.9282382167072406
(-1000.0, 119102.0)
[4, 3, 2, 1, 0, 6]
#ffffb3
0.13100211228024514
#bebada
0.1621581092098728
#fb8072
0.21688096617404234
#80b1d3
0.30314894323988
#fdb462
0.32849265164700725
grey
0.9282382167072406
(-1000.0, 119102.0)
[5, 4, 3, 2, 1, 0, 6]
#8dd3c7
0.12671029902108336
#ffffb3
0.13100211228024514
#bebada
0.1621581092098728
#fb8072
0.21688096617404234
#80b1d3
0.30314894323988
#fdb462
0.32849265164700725
grey
0.9282382167072406
(-1000.0, 119102.0)


In [10]:
means

nmeans

array([0.26134192, 0.23346384, 0.13856906, 0.07837392, 0.04410232,
       0.03938133, 0.92106204])

In [11]:
for x in xs:
    print(X[x].sum(0).mean())
    print(X[x].sum(0).mean()/X[x].shape[0]*100)
    print(X[x].shape)
    print('\n\n')
    
    

169.52702077667388
0.13207462061023076
(128357, 108102)



158.94852169017744
0.14679804731399784
(108277, 92369)



115.27574817335602
0.2619903367576273
(44000, 49955)



87.34006077202986
0.4663608541864046
(18728, 26657)



52.82280292190677
0.7610258308875777
(6941, 13279)



14.361679604798871
0.7771471647618436
(1848, 2834)



53.88051022322266
4.531582020456069
(1189, 10662)





In [12]:
for x in xs:
    print(X[x].getnnz(0).mean())
    print(X[x].getnnz(0).mean()/X[x].shape[0]*100)
    print(X[x].shape)
    print('\n\n')
    

112.20582412906329
0.08741698865590758
(128357, 108102)



105.99933960527883
0.09789645040523734
(108277, 92369)



76.54727254529077
0.17397107396656994
(44000, 49955)



57.605394455490114
0.3075896756487084
(18728, 26657)



35.970555011672566
0.5182330357538188
(6941, 13279)



10.720536344389556
0.580115603051383
(1848, 2834)



24.309510410804727
2.0445340967876136
(1189, 10662)





In [13]:
X_y.sum(0).mean()


53.88051022322266